In [1]:
import torch
import gc

print("=== GPU 상태 확인 ===")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
print(f"현재 장치: {torch.cuda.current_device()}")
print(f"GPU 개수: {torch.cuda.device_count()}")

for i in range(torch.cuda.device_count()):
    print(f"\nGPU {i}:")
    print(f"  이름: {torch.cuda.get_device_name(i)}")
    print(f"  메모리 사용량: {torch.cuda.memory_allocated(i)/1e9:.2f} GB")
    print(f"  메모리 예약량: {torch.cuda.memory_reserved(i)/1e9:.2f} GB")
    print(f"  전체 메모리: {torch.cuda.get_device_properties(i).total_memory/1e9:.2f} GB")

=== GPU 상태 확인 ===
CUDA 사용 가능: True
현재 장치: 0
GPU 개수: 4

GPU 0:
  이름: NVIDIA GeForce RTX 4070 Ti SUPER
  메모리 사용량: 0.00 GB
  메모리 예약량: 0.00 GB
  전체 메모리: 16.72 GB

GPU 1:
  이름: NVIDIA GeForce RTX 4070 Ti SUPER
  메모리 사용량: 0.00 GB
  메모리 예약량: 0.00 GB
  전체 메모리: 16.72 GB

GPU 2:
  이름: NVIDIA RTX A4000
  메모리 사용량: 0.00 GB
  메모리 예약량: 0.00 GB
  전체 메모리: 16.75 GB

GPU 3:
  이름: NVIDIA RTX A4000
  메모리 사용량: 0.00 GB
  메모리 예약량: 0.00 GB
  전체 메모리: 16.75 GB


In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [3]:
import argparse
import os
import sys
from pathlib import Path
import json
import torch
from torch.utils.data import DataLoader
from torch.cuda.amp import autocast, GradScaler
from tqdm.auto import tqdm
import sqlite3
import subprocess
from datetime import datetime

# 프로젝트 루트 디렉토리를 Python 경로에 추가합니다.
# 이렇게 해야 dataset, models 등 서브 디렉토리의 모듈을 직접 임포트할 수 있습니다.
# 현재 Jupyter Notebook이 MANAGER 디렉토리 내에서 실행된다고 가정합니다.
project_root = Path(os.getcwd())
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

# 커스텀 모듈 임포트
from dataset.data import VideoPersonDataset
from models.manager_graphtokens import GraphTokenManager
from experiments import init_experiment_db, insert_new_experiment, update_experiment_metrics, insert_sample_predictions
# train.py, validate.py, eval.py의 핵심 함수를 직접 임포트합니다.
# argparse 없이 함수 인자로 직접 값을 전달할 수 있게 됩니다.
from train import seed_all, get_dataloaders, evaluate as train_evaluate # train.py의 evaluate는 이름 충돌 피하기 위해 train_evaluate로 변경
from validate import run_eval as validate_run_eval, load_ckpt as validate_load_ckpt, get_loader as validate_get_loader
from eval import run_test as eval_run_test, load_ckpt as eval_load_ckpt # eval.py의 load_ckpt도 이름 충돌 피하기 위해 변경

print("모든 모듈 임포트 완료 및 경로 설정 완료.")

/home/dsl/anaconda3/envs/chatglm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/dsl/anaconda3/envs/chatglm/lib/python3.11/site-packages/torch_geometric/typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /home/dsl/anaconda3/envs/chatglm/lib/python3.11/site-packages/torch_scatter/_version_cuda.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSsb
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/home/dsl/anaconda3/envs/chatglm/lib/python3.11/site-packages/torch_geometric/typing.py:97: UserWarning: An issue occurred while importing 'torch-cluster'. Disabling its usage. Stacktrace: /home/dsl/anaconda3/envs/chatglm/lib/python3.11/site-packages/torch_cluster/_version_cuda.so: undefined symbol: _ZN5torch3jit17parseS

모든 모듈 임포트 완료 및 경로 설정 완료.


In [4]:
import pandas as pd

# ready_videos.csv 에 “video_id” 라는 컬럼이 있다고 가정
ready_df = pd.read_csv("data/ready_videos.csv")
valid_video_ids = set(ready_df["video_id"].astype(str))  # 문자열로 통일

In [5]:
# 실험 설정 (하이퍼파라미터 및 경로)
class ExperimentArgs:
    def __init__(self):
        self.db = "data/speech_segments.db" # 데이터베이스 경로
        self.cache = "cache/" # 캐시 디렉토리
        self.frames = "data/frames/" # 비디오 프레임 경로
        self.wav = "data/wav/" # 오디오 파일 경로
        self.ckpt_dir = "checkpoints/" # 체크포인트 저장 디렉토리
        self.epochs = 5 # 훈련 에포크 수
        self.lr = 2e-4 # 학습률
        self.seed = 42 # 랜덤 시드
        self.max_samples = None # 디버깅용 샘플 제한 (전체 데이터 사용 시 None)
        self.output_csv = "results_test.csv" # eval.py에서 개별 예측 저장할 CSV
        self.output_json = "metrics_test.json" # eval.py에서 최종 메트릭 저장할 JSON

args = ExperimentArgs() # 인자 객체 생성

# 체크포인트 디렉토리 생성
Path(args.ckpt_dir).mkdir(exist_ok=True)

# 실험 데이터베이스 초기화 및 연결
experiment_db_path = "experiment_results.db" # 실험 결과를 저장할 DB 파일명
exp_conn = init_experiment_db(db_path=experiment_db_path)
print(f"실험 데이터베이스 '{experiment_db_path}' 초기화 및 연결 완료.")

# 새 실험 레코드 삽입 (초기 정보)
experiment_id = insert_new_experiment(exp_conn, args)
print(f"새로운 실험이 ID: {experiment_id}로 시작되었습니다.")

실험 데이터베이스 'experiment_results.db' 초기화 및 연결 완료.
새로운 실험이 ID: 8로 시작되었습니다.


In [6]:
# 훈련 장치 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"훈련 장치: {device}")

# 랜덤 시드 설정
seed_all(args.seed)

# 데이터 로더 준비
train_loader, val_loader = get_dataloaders(
    db=args.db, cache=args.cache, frames=args.frames, wav=args.wav,
    batch=1, max_samples=args.max_samples, seed=args.seed
)
print(f"훈련 샘플: {len(train_loader.dataset)}, 검증 샘플: {len(val_loader.dataset)}")

# 모델 로드 및 GPU 이동
model = GraphTokenManager()
model.train()

# 옵티마이저 설정
optim = torch.optim.AdamW(
    model.parameters(), lr=args.lr, betas=(0.9,0.95), eps=1e-8
)
scaler = GradScaler()

best_f1 = 0.0
current_best_ckpt_path = None # 최적 체크포인트 경로 추적

print("모델 훈련 시작...")
for epoch in range(1, args.epochs + 1):
    model.train()
    pbar = tqdm(train_loader, desc=f"Epoch {epoch} (Train)")
    for sample in pbar:
        g = sample["graph"].to(device)
        lbl = sample["label"].float().unsqueeze(0).to(device)

        optim.zero_grad()
        with autocast():
            logit, loss = model(g, sample["person"][0], lbl)
        scaler.scale(loss).backward()
        scaler.unscale_(optim)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(optim); scaler.update()

        pbar.set_postfix(loss=loss.item())

    # ── 검증 (Validation) ──
    # train.py의 evaluate 함수는 F1만 반환하지만, precision, recall도 필요하므로
    # validate.py의 run_eval을 사용하거나 train_evaluate를 수정해야 합니다.
    # 여기서는 validate.py의 run_eval을 사용하겠습니다.
    val_metrics = validate_run_eval(val_loader, model, device) # run_eval은 dict 반환

    print(f"Epoch {epoch} Val Metrics: {json.dumps(val_metrics, indent=2)}")

    # DB에 검증 성능 업데이트
    update_experiment_metrics(exp_conn, experiment_id, "val", val_metrics)
    print(f"실험 {experiment_id}의 검증 메트릭이 DB에 업데이트되었습니다.")

    # 최적 모델 저장
    if val_metrics['f1'] > best_f1:
        best_f1 = val_metrics['f1']
        current_best_ckpt_path = Path(args.ckpt_dir) / f"best_exp{experiment_id}.pt" # 실험 ID를 포함하여 저장
        torch.save({
            "gcn" : model.gcn.state_dict(),
            "proj": model.proj_up.state_dict(),
            "lora": model.glm.state_dict(),
            "optim": optim.state_dict(),
        }, current_best_ckpt_path)
        print(f"  ✔ 새로운 최적 체크포인트 저장됨 (F1 {best_f1:.4f} at {current_best_ckpt_path})")

# 최종 체크포인트 경로를 DB에 업데이트 (best.pt가 아닌 고유 경로)
if current_best_ckpt_path:
    # 이전에 best_f1 업데이트 시 checkpoint_path도 함께 업데이트 되도록 experiments.py 수정했으므로 중복될 수 있습니다.
    # 필요하다면 여기서 한번 더 최종 업데이트를 강제할 수 있습니다.
    pass # 이미 위에서 F1 업데이트 시 경로도 업데이트되므로 생략
else:
    print("최적 체크포인트가 저장되지 않았습니다.")

print("모델 훈련 완료.")

훈련 장치: cuda


/home/dsl/anaconda3/envs/chatglm/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 7/7 [00:07<00:00,  1.11s/it]
/home/dsl/anaconda3/envs/chatglm/lib/python3.11/site-packages/transformers/utils/deprecation.py:165: UserWarning: The following named arguments are not valid for `BeitImageProcessor.__init__` and were ignored: 'feature_extractor_type'
  return func(*args, **kwargs)
Some weights of the model checkpoint at facebook/hubert-base-ls960 were not used when initializing HubertModel: ['encoder.pos_conv_embed.conv.weight_g', 'encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing HubertModel f

훈련 샘플: 15735, 검증 샘플: 2320


Loading checkpoint shards: 100%|██████████| 7/7 [00:08<00:00,  1.24s/it]
/tmp/ipykernel_3887285/1274914417.py:23: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


모델 훈련 시작...


...

DEBUG: Received script with type: <class 'str'>, value: 'So when you want to give like an interview after,'
DEBUG: Received script with type: <class 'str'>, value: 'you know, give a post-mortem,'
DEBUG: Received script with type: <class 'str'>, value: 'you think you would go to a traditional media outlet'
DEBUG: Received script with type: <class 'str'>, value: 'that way they can ask some, you know, tough questions.'
DEBUG: Received script with type: <class 'str'>, value: 'Instead, though, Obama crew that ran Kamala Harris campaign'
DEBUG: Received script with type: <class 'str'>, value: 'went to the Obama bros,'
DEBUG: Received script with type: <class 'str'>, value: 'which were the speech writers, the comms team.'
DEBUG: Received script with type: <class 'str'>, value: 'Hi, save America.'
DEBUG: Received script with type: <class 'str'>, value: 'Hi, save America, and this is what they have to say.'
DEBUG: Received script with type: <class 'str'>, value: 'We would do an interview,'
DEBU

/tmp/ipykernel_3887285/1274914417.py:37: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 1 (Train):   0%|          | 1/15735 [00:51<224:43:51, 51.42s/it, loss=nan]/home/dsl/anaconda3/envs/chatglm/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 7/7 [00:08<00:00,  1.27s/it]


DEBUG: Received script with type: <class 'str'>, value: ' Lev Parnas was the centerpiece of a bomb throwing congressional hearing on the Biden impeachment inquiry this week.'
DEBUG: Received script with type: <class 'str'>, value: 'Love Parnas was the centerpiece of a bomb throwing congressional hearing on the Biden impeachment inquiry this week.'
DEBUG: Received script with type: <class 'str'>, value: ' Joining me now for his first extended interview since that feisty hearing is Lev Parnas himself.'
DEBUG: Received script with type: <class 'str'>, value: 'Joining me now for his first extended interview since that feisty hearing is Love Parnas himself.'
DEBUG: Received script with type: <class 'str'>, value: ' Mr. Parnas was a former associate of Rulie Giuliani's.'
DEBUG: Received script with type: <class 'str'>, value: 'Mr. Parnas was a former associate of Rulie Giuliani's.'
DEBUG: Received script with type: <class 'str'>, value: ' He was also convicted of campaign finance crimes, wir

/tmp/ipykernel_3887285/1274914417.py:37: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 1 (Train):   0%|          | 1/15735 [01:50<484:54:16, 110.95s/it, loss=nan]


OutOfMemoryError: CUDA out of memory. Tried to allocate 11.40 GiB. GPU 3 has a total capacity of 15.60 GiB of which 2.60 GiB is free. Including non-PyTorch memory, this process has 12.95 GiB memory in use. Of the allocated memory 12.35 GiB is allocated by PyTorch, and 396.24 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
print("\n테스트 세트 평가 시작...")

# 최적 체크포인트 경로가 없으면 에러 또는 기본값 사용
if not current_best_ckpt_path or not current_best_ckpt_path.exists():
    print(f"[WARN] 최적 체크포인트 '{current_best_ckpt_path}'를 찾을 수 없습니다. 기본 체크포인트 사용 시도.")
    # 실제 환경에서는 여기서 오류 처리하거나 기본 ckpt 경로로 대체해야 합니다.
    # 예시: current_best_ckpt_path = Path(args.ckpt_dir) / "best.pt"

if current_best_ckpt_path and current_best_ckpt_path.exists():
    # 모델 재로드 (최적 체크포인트 로드)
    test_model = GraphTokenManager().half().to(device)
    eval_load_ckpt(test_model, current_best_ckpt_path) # eval.py의 load_ckpt 함수 사용
    print(f"테스트 평가를 위해 모델에 '{current_best_ckpt_path}' 체크포인트 로드 완료.")

    # 테스트 데이터 로더 준비
    test_loader = validate_get_loader( # validate.py의 get_loader 사용
        db=args.db, cache=args.cache, frames=args.frames, wav=args.wav,
        split="test", max_samples=args.max_samples, seed=args.seed
    )
    print(f"테스트 샘플: {len(test_loader.dataset)}")

    # 최종 평가 실행 (eval.py의 run_test 사용)
    # run_test는 메트릭과 개별 예측 리스트를 반환합니다.
    final_metrics, individual_predictions_list = eval_run_test(test_loader, test_model, device, Path(args.output_csv))
    print(f"\n최종 테스트 메트릭: {json.dumps(final_metrics, indent=2)}")

    # DB에 최종 테스트 성능 업데이트
    update_experiment_metrics(exp_conn, experiment_id, "test", final_metrics)
    print(f"실험 {experiment_id}의 최종 테스트 메트릭이 DB에 업데이트되었습니다.")

    # 개별 샘플 예측 결과 DB에 저장
    insert_sample_predictions(exp_conn, experiment_id, individual_predictions_list, "test")
    print(f"{len(individual_predictions_list)}개의 개별 샘플 예측 결과가 DB에 저장되었습니다.")

    # 최종 메트릭 JSON 파일 저장
    Path(args.output_json).write_text(json.dumps(final_metrics, indent=2))
    print(f"최종 결과가 {args.output_csv} 및 {args.output_json}에 저장되었습니다.")

else:
    print("[ERROR] 테스트 평가를 위한 최적 체크포인트를 찾을 수 없습니다. 평가를 건너뜝니다.")

# 데이터베이스 연결 종료
exp_conn.close()
print("모든 실험 과정 완료 및 데이터베이스 연결 종료.")